In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

all_data = pd.read_csv('sales所在周次节假日_剔除换货.csv')
all_data.rename(columns={'店铺名称':'store','地理位置指数':'position','年份':'years','周次':'weeks of year','是否包含法定节假日':'Isholiday','季节':'season','大类':'item','销量(剔除换货）':'sales','均价（剔除换货）':'prices','促销（剔除换货）':'discount'},inplace=True)
all_data['weeks'] = all_data['weeks of year']

In [2]:
a = all_data
week_5 = a.loc[lambda a : a['years'] == 2015]
week_6 = a.loc[lambda a : a['years'] == 2016]
week_6['weeks'] += 53
a = pd.concat([week_5,week_6])

/home/lin/anaconda3/envs/boost/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
from sklearn import preprocessing

#map店铺名称
name = a['store']
le = preprocessing.LabelEncoder()
le.fit(name)
list(name)
le.transform(name)
tmp = le.transform(name)
a['store'] = tmp

#map合并大类
category = a['item']
le.fit(category)
list(category)
le.transform(category)
tmp = le.transform(category)
a['item'] = tmp

#map季节
season = a['season']
le.fit(season)
list(season)
le.transform(season)
tmp = le.transform(season)
a['season'] = tmp


In [4]:
df = a
masked_series = (df['years']==2016) & (df['weeks'].isin([53,54,55]))
masked_series2 = (df['years']==2016) & (~(df['weeks'].isin([53,54,55])))
df.loc[(masked_series), 'train_or_test'] = 'test'
df.loc[(masked_series2), 'train_or_test'] = 'no_train'
print('Train shape: {}'.format(df.loc[df['years']==2015,:].shape))
print('Test shape: {}'.format(df.loc[df.train_or_test=='test',:].shape))


Train shape: (8119, 12)
Test shape: (340, 12)


In [5]:
df_n = df.drop(['train_or_test'],axis=1)


In [6]:
df_n['sales'] = np.log1p(df.sales.values)
df_n['prices'] = np.log1p(df.sales.values)
df_n.sample(2)

,store,position,years,weeks of year,Isholiday,season,item,sales,prices,discount,weeks
4769,7,7,2016,34,0,1,3,1.609438,1.609438,0.229287,87
17961,29,7,2016,24,1,3,7,1.386294,1.386294,0.328820,77


In [7]:
''''
def one_hot_encoder(df_n, ohe_cols=['store','item','weeks of years','isholiday','season','item']):

#    One-Hot Encoder function
    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))
    df_n = pd.get_dummies(df_n, columns=ohe_cols)
    print('New df shape:{}'.format(df.shape))
    return df_n
'''

"'\ndef one_hot_encoder(df_n, ohe_cols=['store','item','weeks of years','isholiday','season','item']):\n\n#    One-Hot Encoder function\n    print('Creating OHE features..\nOld df shape:{}'.format(df.shape))\n    df_n = pd.get_dummies(df_n, columns=ohe_cols)\n    print('New df shape:{}'.format(df.shape))\n    return df_n\n"

In [8]:
train = df_n[lambda df_n: df_n['years']==2015]
t1 = df_n[lambda df_n: df_n['weeks']==54]
t2 = df_n[lambda df_n: df_n['weeks']==55]
t3 = df_n[lambda df_n: df_n['weeks']==56]
test = pd.concat([t1,t2,t3])

In [9]:
#train = one_hot_encoder(train, ohe_cols=['store','item','weeks of year','Isholiday','season','item']) 

In [10]:
y_train = train['sales']
y_test = test['sales']
X_train = train.drop('sales', axis=1)
X_test = test.drop('sales', axis=1)

In [11]:
import lightgbm as lgb
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [12]:
from sklearn.metrics import mean_squared_error
print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)


Starting training...
[1]	valid_0's l1: 0.705466	valid_0's l2: 0.742124
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.670139	valid_0's l2: 0.669488
[3]	valid_0's l1: 0.636546	valid_0's l2: 0.603795
[4]	valid_0's l1: 0.604659	valid_0's l2: 0.544624
[5]	valid_0's l1: 0.574333	valid_0's l2: 0.491114
[6]	valid_0's l1: 0.545544	valid_0's l2: 0.442906
[7]	valid_0's l1: 0.518228	valid_0's l2: 0.399543
[8]	valid_0's l1: 0.49222	valid_0's l2: 0.36023
[9]	valid_0's l1: 0.467532	valid_0's l2: 0.324824
[10]	valid_0's l1: 0.44404	valid_0's l2: 0.292794
[11]	valid_0's l1: 0.421778	valid_0's l2: 0.264049
[12]	valid_0's l1: 0.400616	valid_0's l2: 0.23808
[13]	valid_0's l1: 0.38053	valid_0's l2: 0.214705
[14]	valid_0's l1: 0.361419	valid_0's l2: 0.19355
[15]	valid_0's l1: 0.343264	valid_0's l2: 0.174485
[16]	valid_0's l1: 0.326067	valid_0's l2: 0.157381
[17]	valid_0's l1: 0.309727	valid_0's l2: 0.141943
[18]	valid_0's l1: 0.29421	valid_0's l2: 0.128041
[19]	valid_0's l